In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import pandas as pd
import plotly as px
sys.path.append(os.path.abspath(os.path.join(os.pardir)))

In [3]:
from src.data import load_raw_data

In [4]:
rides = load_raw_data(year=2022)
rides

File 2022-01 is already available in storage
File 2022-02 is already available in storage
File 2022-03 is already available in storage
File 2022-04 is already available in storage
File 2022-05 is already available in storage
File 2022-06 is already available in storage
File 2022-07 is already available in storage
File 2022-08 is already available in storage
File 2022-09 is already available in storage
File 2022-10 is already available in storage
File 2022-11 is already available in storage
File 2022-12 is already available in storage


,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
3399544,2022-12-31 23:46:00,16
3399545,2022-12-31 23:13:24,75
3399546,2022-12-31 23:00:49,168
3399547,2022-12-31 23:02:50,238


In [5]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)
ts_data

100%|██████████| 262/262 [00:03<00:00, 74.26it/s]


,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4
...,...,...,...
2295115,2022-12-31 19:00:00,0,199
2295116,2022-12-31 20:00:00,0,199
2295117,2022-12-31 21:00:00,0,199
2295118,2022-12-31 22:00:00,0,199


In [6]:
ts_data.pickup_location_id.unique().size

262

In [7]:
from src.data import transform_ts_data_into_features_and_target
features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_sequence_len=24*28*1, ## USing one month of history
    step_size=24
)
print(f'{features.shape=}')
print(f'{targets.shape=}')

  0%|          | 0/262 [00:00<?, ?it/s]

100%|██████████| 262/262 [00:25<00:00, 10.20it/s]

features.shape=(88294, 674)
targets.shape=(88294,)


In [8]:
features.pickup_location_id.unique().size

262

In [9]:
tarbular_data = features
tarbular_data['target_rides_next_hour'] = targets

from src.paths import TRANSFORMED_DATA_DIR
tarbular_data.to_parquet(TRANSFORMED_DATA_DIR / 'tarbular_data.parquet')

In [10]:
features_and_targets = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tarbular_data.parquet')

In [11]:
features_and_targets.head(10)

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hours,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,4.0,3.0,...,6.0,2.0,3.0,2.0,6.0,4.0,6.0,2022-02-03,4,6.0
6,3.0,2.0,4.0,0.0,1.0,0.0,2.0,5.0,1.0,0.0,...,5.0,11.0,12.0,3.0,3.0,1.0,13.0,2022-02-04,4,3.0
7,16.0,6.0,8.0,4.0,0.0,0.0,0.0,3.0,2.0,1.0,...,9.0,9.0,9.0,4.0,7.0,7.0,19.0,2022-02-05,4,31.0
8,18.0,23.0,12.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,...,16.0,12.0,16.0,16.0,7.0,21.0,31.0,2022-02-06,4,43.0
9,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,6.0,3.0,...,4.0,8.0,1.0,2.0,2.0,1.0,0.0,2022-02-07,4,0.0
